In [ ]:
import numpy as np
import tiledb

In [ ]:
ctx = tiledb.Ctx()

In [ ]:
xdim = tiledb.Dim(ctx, 'X', domain=(0, 3), dtype=np.int32, tile=4)
ydim = tiledb.Dim(ctx, 'Y', domain=(0, 9), dtype=np.int32, tile=5)

In [ ]:
xdim.shape, ydim.shape

In [ ]:
dom = tiledb.Domain(ctx, xdim, ydim)

In [ ]:
dom.shape, dom.ndim, dom.dim(0)

In [ ]:
dom2 = tiledb.Domain(ctx, xdim, ydim, ydim)

In [ ]:
dom2.shape

In [ ]:
p = tiledb.Attr(ctx, name='pressure', dtype=np.float32)
u = tiledb.Attr(ctx, name='u', dtype=np.float32)
v = tiledb.Attr(ctx, name='v', dtype=np.float32)

In [ ]:
p.dtype

In [ ]:
p.name

In [ ]:
schema = tiledb.ArraySchema(ctx, domain=dom, sparse=False,
                            attrs=[p, u, v])

In [ ]:
schema.shape

In [ ]:
for i in range(schema.nattr):
    print(schema.attr(i).name)

In [ ]:
aname = 'foo1'

In [ ]:
tiledb.DenseArray.create(aname, schema)

In [ ]:
data = np.zeros((4, 10), dtype=np.float32)

In [ ]:
data.shape

In [ ]:
A = tiledb.DenseArray(ctx, aname, mode='w')

In [ ]:
A[:] = {'pressure': data, 'u': data, 'v': data}

In [ ]:
A.close()

In [ ]:
with tiledb.DenseArray(ctx, aname, mode='w') as A:
    A[:] = {'pressure': data, 'u': data, 'v': data}

In [ ]:
with tiledb.DenseArray(ctx, aname, mode='r') as A:
    d = A[0:4,0:10]
    print(d['pressure'])

In [ ]:
with tiledb.DenseArray(ctx, aname, mode='w') as A:
    A[1:3,2:7] = {'pressure': 1 + data[1:3, 2:7],
                  'u': -1 + data[1:3, 2:7], 
                  'v': 3 + data[1:3, 2:7]}

In [ ]:
with tiledb.DenseArray(ctx, aname, mode='r') as A:
    d = A[:]
    print(d['pressure'])
    print(d['u'])
    print(d['v'])

In [ ]:
with tiledb.DenseArray(ctx, aname, mode='r') as A:
    d = A.query(attrs=['pressure'])[1, 2:5]
    print(d['pressure'])

In [ ]:
bname = 'bar1'

In [ ]:
tiledb.DenseArray.create(bname, schema)

In [ ]:
with tiledb.DenseArray(ctx, bname, mode='w') as A:
    A[1:3, 4:8] = {'pressure': data[1:3, 4:8], 
                   'u': data[1:3, 4:8], 
                   'v': data[1:3, 4:8]}

In [ ]:
with tiledb.DenseArray(ctx, bname, mode='r') as A:
    d = A.query(attrs=['pressure'])[:]
    print(d['pressure'])

In [ ]:
kvname = 'kvname'
kvschema = tiledb.KVSchema(ctx, 
                           attrs=[tiledb.Attr(ctx, name="a1", 
                                              dtype=bytes)])
kv = tiledb.KV.create(ctx, kvname, kvschema)

In [ ]:
with tiledb.KV(ctx, kvname, mode='w') as KV:
    KV['foo'] = 'foo!'
    KV['ss.dd'] = '0.33'

In [ ]:
with tiledb.KV(ctx, kvname, mode='r') as KV:
    for k in KV:
        print(k)
   